## Задание 2.1


Промежуточная задача - преобразовать данные в pandas.DataFrame вида {user, item, order}, где order - порядковый номер с конца (0 - самый "свежий" лайк, чем больше order, тем позже был поставлен лайк)

**Итоговая задача** - построить схему валидации для данного соревнования с учетом особенностей соревнования
- Между `train` и `test` не должно быть общих пользователей
- Количество фолдов задается через параметр класса `n_folds`
- В `test` должно быть не более `p` последних треков (параметр класса `p`)

##1.Промежуточная задача

In [1]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
from sklearn.model_selection import KFold

In [2]:
user_id_col = []
item_id_col = []

with open('/content/drive/MyDrive/Yandex Cup/train') as file:
    
    rows = file.readlines()
    
    for index, row in enumerate(tqdm(range(len(rows)))):
        likes = [int(i) for i in rows[row].strip().split(' ')][::-1]
        user_id = [index] * len(likes)
        user_id_col.extend(user_id)
        item_id_col.extend(likes)

  0%|          | 0/1160084 [00:00<?, ?it/s]

In [3]:
df = pd.DataFrame({
    'user_id': user_id_col,
    'item_id': item_id_col
})

In [4]:
df.to_csv('/content/drive/MyDrive/Yandex Cup/train_improved.csv', index=False)

In [2]:
df = pd.read_csv('/content/drive/MyDrive/Yandex Cup/train_improved.csv')

In [3]:
df

,user_id,item_id
0,0,388242
1,0,278503
2,0,102795
3,0,470957
4,0,159637
...,...,...
94188629,1160083,19120
94188630,1160083,326821
94188631,1160083,214132
94188632,1160083,352098


In [4]:
# Добавление порядкового столбца order по "свежести" лайков
df['order'] = df.groupby('user_id').cumcount() 

In [5]:
df

,user_id,item_id,order
0,0,388242,0
1,0,278503,1
2,0,102795,2
3,0,470957,3
4,0,159637,4
...,...,...,...
94188629,1160083,19120,251
94188630,1160083,326821,252
94188631,1160083,214132,253
94188632,1160083,352098,254


##2.Итоговая задача

In [6]:
class UsersKFoldPOut():
    def __init__(self, n_folds: int, p: int, random_seed: int=23):
        self.n_folds = n_folds
        self.p = p
        self.random_seed = random_seed
    
    def split(self, df: pd.DataFrame):
        users = df['user_id'].unique()
        kfold = KFold(n_splits=self.n_folds, shuffle=True, random_state=self.random_seed)

        for train_samples, test_samples in kfold.split(users):
            train_mask = df['user_id'].isin(train_samples)
            test_mask = df['user_id'].isin(test_samples) & (df['order'] < self.p)
            yield train_mask, test_mask

In [7]:
n_folds = 3
p = 1

cv = UsersKFoldPOut(n_folds=n_folds, p=p)

for i, (train_mask, test_mask) in enumerate(cv.split(df)):
     train = df[train_mask]
     test = df[test_mask]
     
     # Проверка на отсутствие общих пользователей в трейне и тесте
     assert len(set(train['user_id'].unique()).intersection(test['user_id'].unique())) == 0, 'Common users found in train and test'
     # Проверка, что в тесте не более p последних треков
     assert test.groupby('user_id').count().values.max(), 'More than p latest tracks in the test'

     print(f'Fold#{i} | Train: {train.shape[0]}, Test: {test.shape[0]}')

Fold#0 | Train: 62769950, Test: 386695
Fold#1 | Train: 62749697, Test: 386695
Fold#2 | Train: 62857621, Test: 386694
